In [1]:
# Load Azure Libraries

from azureml.core import Workspace,Dataset,Experiment
from azureml.core.compute import ComputeTarget
from azureml.train.automl import AutoMLConfig
from azureml.train.automl.run import AutoMLRun
from azureml.widgets import RunDetails

#Load non-azure libraries

import pandas as pd
import numpy as np

In [2]:
# Retrieve the workspace

ws = Workspace.from_config()

# Retrieve compute target to run automl remotely

compute_name = 'compute-cluster'
compute_target = ComputeTarget(ws,compute_name)

In [5]:
from azureml.core import Datastore

In [8]:
# Retrieve the datastore

datastore = Datastore.get_default(ws)
my_datastore_name = 'workspaceblobstore'
my_datastore = Datastore.get(ws,my_datastore_name)

In [10]:
# retrieve Dataset

dataset_name = "Diabetes_Sample"
dataset = Dataset.get_by_name(ws,dataset_name,version='latest')

In [11]:
dataset.take(10).to_pandas_dataframe()

,AGE,SEX,BMI,BP,S1,S2,S3,S4,S5,S6,Y
0,59,2,32.1,101.0,157,93.2,38.0,4.00,4.8598,87,151
1,48,1,21.6,87.0,183,103.2,70.0,3.00,3.8918,69,75
2,72,2,30.5,93.0,156,93.6,41.0,4.00,4.6728,85,141
3,24,1,25.3,84.0,198,131.4,40.0,5.00,4.8903,89,206
4,50,1,23.0,101.0,192,125.4,52.0,4.00,4.2905,80,135
5,23,1,22.6,89.0,139,64.8,61.0,2.00,4.1897,68,97
6,36,2,22.0,90.0,160,99.6,50.0,3.00,3.9512,82,138
7,66,2,26.2,114.0,255,185.0,56.0,4.55,4.2485,92,63
8,60,2,32.1,83.0,179,119.4,42.0,4.00,4.4773,94,110
9,29,1,30.0,85.0,180,93.4,43.0,4.00,5.3845,88,310


In [14]:
# Create a name for automl

experiment_name = 'Diabetes-Sample-Regression'

# Set the experiment to the workspace

exp = Experiment(workspace = ws,name=experiment_name)

In [18]:
# Define the variables

target_column = 'Y'
task = 'regression'
primary_metric = 'normalized_root_mean_squared_error'
featurization ='auto'

In [19]:
# Set the autoconfig automl

config = AutoMLConfig(task=task,
                     primary_metric=primary_metric,
                     featurization=featurization,
                     compute_target=compute_target,
                     training_data=dataset,
                     label_column_name=target_column,
                     experiment_timeout_minutes=15,
                     enable_early_stopping=True,
                     n_cross_validations=5,
                     model_explainability=True)

In [20]:
# Train the diabetes dataset

AutoML_run = exp.submit(config,show_output=True)
RunDetails(AutoML_run).show()

Submitting remote run.
No run_configuration provided, running on compute-cluster with default configuration
Running on remote compute: compute-cluster


Experiment,Id,Type,Status,Details Page,Docs Page
Diabetes-Sample-Regression,AutoML_00f2d3cd-a40d-4cf6-bb60-96c84ca28117,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

********************************************************************************************

TYPE:         High cardinality feature detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and no high cardinality features were detected.
              Learn more about high cardinality feature handling: https://aka.ms/AutomatedMLFeaturization

************************************************************

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…